# Batch Read Example
Read a set of records, if record is not found, return null.

### Housekeeping - Setup & wipe out any prior records on the Aerospike Server

We have a namespace **_test_** pre-defined on the server. Lets truncate it using _asadm_.

This is needed while doing code development. If you want to clear the iJava Kernel of all Java objects and run all cells from scratch, Kernel->Restart & Run All, this will ensure any records written on the underlying Aerospike cluster are purged.

First, we need required imports for using %sh in interactive Java Kernel. (This is specific to the iJava Kernel implementation by **Spencer Park** that we are using.)

In [1]:
import io.github.spencerpark.ijava.IJava;
import io.github.spencerpark.jupyter.kernel.magic.common.Shell;
IJava.getKernelInstance().getMagics().registerMagics(Shell.class);

### Running _asadm_ in iJava
We can run _asadm_ commands inline. Below, we will use the truncation command, which normally requires an interactive confirmation, which we will skip by using the _--no-warn_ flag. No output will be displayed.

In [2]:
%sh asadm --enable -e "manage truncate ns test --no-warn" -h "127.0.0.1"

#### Add Java Client POM Dependency  
Jupyter Notebook way!

In [3]:
%%loadFromPOM
<dependencies>
  <dependency>
    <groupId>com.aerospike</groupId>
    <artifactId>aerospike-client-jdk8</artifactId>
    <version>8.1.1</version>
  </dependency>
</dependencies>

####  Java Client Imports and Test code

In [4]:
//Required Imports
import com.aerospike.client.AerospikeClient;
import com.aerospike.client.policy.WritePolicy;
import com.aerospike.client.Bin;
import com.aerospike.client.Key;
import com.aerospike.client.Record;
import com.aerospike.client.Value;

import com.aerospike.client.policy.BatchPolicy;
import com.aerospike.client.ResultCode;


In [5]:
AerospikeClient client = new AerospikeClient("127.0.0.1", 3000);


In [6]:
Key key1 = new Key("test", "testSet1", 1); 
Key key2 = new Key("test", "testSet2", 2);  // We will not create this record 
Key key3 = new Key("test", "testSet3", 3); 
Key key4 = new Key("test", "testSet4", 4);
Bin b1 = new Bin("bin1", 100);
Bin b2 = new Bin("bin1", 200);
Bin b3 = new Bin("bin1", 300);
Bin b4 = new Bin("bin1", 400);
client.put(null, key1, b1);
//client.put(null, key2, b2); //Skipping so its a null record
client.put(null, key3, b3);
client.put(null, key4, b4);


In [7]:
System.out.println(client.get(null, key1));
System.out.println(client.get(null, key2));
System.out.println(client.get(null, key3));
System.out.println(client.get(null, key4));

(gen:1),(exp:453261759),(bins:(bin1:100))
null
(gen:1),(exp:453261759),(bins:(bin1:300))
(gen:1),(exp:453261759),(bins:(bin1:400))


In [8]:
//Do a batch read
BatchPolicy batchPolicy = new BatchPolicy();

Record [] bReads = client.get(batchPolicy, new Key[] {key1, key2, key3, key4});

// process the batch reads
for (int i = 0; i < bReads.length; i++) {
    Record bRead = bReads[i];    
    if (bRead != null) {          // check individual record
        long bin1Val = bRead.getLong("bin1");         
        System.out.format("Result[%d]: bin1: %d\n",i, bin1Val);
    }
    else {   // error in individual key's operations
        System.out.format("Result[%d]: not found: \n", i);
    }
}


Result[0]: bin1: 100
Result[1]: not found: 
Result[2]: bin1: 300
Result[3]: bin1: 400


# Cleanup

In [ ]:
%sh asadm --enable -e "manage truncate ns test --no-warn" -h "127.0.0.1"